In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset, load_from_disk, concatenate_datasets

from vllm import LLM, SamplingParams
from vllm.sampling_params import GuidedDecodingParams

import json
import re
import os
import math
import pandas as pd

INFO 06-21 17:24:11 [__init__.py:248] Automatically detected platform cuda.


In [2]:
NUM_GPUS = 2
BATCH_SIZE = 128

In [3]:
ds = load_dataset('amang1802/wikipedia_controversial_sections')['train']

In [4]:
ds = ds.filter(lambda row: row['issue_scale'] > 0 and len(row['section_text'].strip()) > 25)

In [5]:
ds

Dataset({
    features: ['article_title', 'url', 'section_title', 'section_text', 'classification_json', 'issue_scale'],
    num_rows: 175575
})

In [6]:
model_id = "google/gemma-3-27b-it"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
llm = LLM(model=model_id, max_model_len=17408, tensor_parallel_size=NUM_GPUS, gpu_memory_utilization=0.9)

INFO 06-21 17:24:14 [__init__.py:30] Available plugins for group vllm.general_plugins:
INFO 06-21 17:24:14 [__init__.py:32] name=lora_filesystem_resolver, value=vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-21 17:24:14 [__init__.py:34] all available plugins for group vllm.general_plugins will be loaded.
INFO 06-21 17:24:14 [__init__.py:36] set environment variable VLLM_PLUGINS to control which plugins to load.
INFO 06-21 17:24:14 [__init__.py:44] plugin lora_filesystem_resolver loaded.
INFO 06-21 17:24:20 [config.py:787] This model supports multiple tasks: {'reward', 'score', 'embed', 'classify', 'generate'}. Defaulting to 'generate'.
INFO 06-21 17:24:20 [config.py:1869] Defaulting to use mp for distributed inference
INFO 06-21 17:24:20 [config.py:2112] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 06-21 17:24:22 [core.py:427] Waiting for init message from front-end.
INFO 06-21 17:24:22 [core.py:61] Initializing a V1 LLM engin

[W621 17:24:24.413613725 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W621 17:24:24.741971261 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W621 17:24:25.177376607 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W621 17:24:25.391554101 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W621 17:24:25.391920084 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3


(VllmWorker rank=1 pid=806528) (VllmWorker rank=0 pid=806527) INFO 06-21 17:24:26 [utils.py:1071] Found nccl from library libnccl.so.2
INFO 06-21 17:24:26 [utils.py:1071] Found nccl from library libnccl.so.2
(VllmWorker rank=1 pid=806528) (VllmWorker rank=0 pid=806527) INFO 06-21 17:24:26 [pynccl.py:69] vLLM is using nccl==2.26.2
INFO 06-21 17:24:26 [pynccl.py:69] vLLM is using nccl==2.26.2


[W621 17:24:26.821802419 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W621 17:24:26.822090599 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3


(VllmWorker rank=1 pid=806528) (VllmWorker rank=0 pid=806527) INFO 06-21 17:24:27 [custom_all_reduce_utils.py:245] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 06-21 17:24:27 [custom_all_reduce_utils.py:245] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorker rank=0 pid=806527) INFO 06-21 17:24:27 [shm_broadcast.py:266] vLLM message queue communication handle: Handle(local_reader_ranks=[1], buffer_handle=(1, 4194304, 6, 'psm_6ae4c5f4'), local_subscribe_addr='ipc:///tmp/10d85176-3e8a-4888-80b5-3cc7f04e83c3', remote_subscribe_addr=None, remote_addr_ipv6=False)
(VllmWorker rank=1 pid=806528) (VllmWorker rank=0 pid=806527) INFO 06-21 17:24:27 [parallel_state.py:1079] rank 0 in world size 2 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
INFO 06-21 17:24:27 [parallel_state.py:1079] rank 1 in world size 2 is assigned as DP rank 0, PP rank 0, TP rank 1, EP rank 1


(VllmWorker rank=1 pid=806528) Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
(VllmWorker rank=0 pid=806527) Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


(VllmWorker rank=0 pid=806527) WARNING 06-21 17:24:32 [topk_topp_sampler.py:58] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(VllmWorker rank=0 pid=806527) INFO 06-21 17:24:32 [gpu_model_runner.py:1503] Starting to load model google/gemma-3-27b-it...
(VllmWorker rank=0 pid=806527) INFO 06-21 17:24:32 [cuda.py:216] Using Flash Attention backend on V1 engine.
(VllmWorker rank=1 pid=806528) WARNING 06-21 17:24:32 [topk_topp_sampler.py:58] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(VllmWorker rank=1 pid=806528) INFO 06-21 17:24:32 [gpu_model_runner.py:1503] Starting to load model google/gemma-3-27b-it...
(VllmWorker rank=0 pid=806527) INFO 06-21 17:24:32 [backends.py:37] Using InductorAdaptor
(VllmWorker rank=1 pid=806528) INFO 06-21 17:24:32 [cuda.py:216] Using 

Loading safetensors checkpoint shards:   0% Completed | 0/12 [00:00<?, ?it/s]


(VllmWorker rank=1 pid=806528) INFO 06-21 17:24:40 [default_loader.py:279] Loading weights took 7.34 seconds
(VllmWorker rank=0 pid=806527) INFO 06-21 17:24:40 [default_loader.py:279] Loading weights took 7.38 seconds
(VllmWorker rank=1 pid=806528) INFO 06-21 17:24:40 [gpu_model_runner.py:1521] Model loading took 25.9044 GiB and 7.692331 seconds
(VllmWorker rank=0 pid=806527) INFO 06-21 17:24:40 [gpu_model_runner.py:1521] Model loading took 25.9044 GiB and 8.068762 seconds
(VllmWorker rank=0 pid=806527) (VllmWorker rank=1 pid=806528) INFO 06-21 17:24:40 [gpu_model_runner.py:1823] Encoder cache will be initialized with a budget of 16384 tokens, and profiled with 64 image items of the maximum feature size.
INFO 06-21 17:24:40 [gpu_model_runner.py:1823] Encoder cache will be initialized with a budget of 16384 tokens, and profiled with 64 image items of the maximum feature size.
(VllmWorker rank=0 pid=806527) INFO 06-21 17:24:55 [backends.py:459] Using cache directory: /root/.cache/vllm/to

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [57]:
system_prompt = """
# Task:
Generate Interview Questions for Opposing Viewpoints on a Divergent Issue

# Instructions
You will be given:

- An excerpt from a Wikipedia article (title, section, and text)
- A divergent_issue_rationale that identifies the core disagreement or debate

Your task is to create two "Explain why..." questions that would elicit responses from people holding opposing viewpoints on exactly one issue. If there is more than one issue, pick only one. These questions should be designed for interviews to understand each side's perspective.

## Guidelines for Creating Questions:

- Identify the two main opposing positions from the text and rationale. Ensure they are about the same issue.
- Frame questions neutrally - avoid loaded language that suggests one side is right.
- Make questions open-ended to encourage detailed explanations.
- Focus on reasoning and values - questions should probe WHY people hold their views.
- Ensure questions are answerable by someone holding that viewpoint.
- Ensure at least one specific detail like name or place from the original text is included in the questions.
- Include the same details in both for and against questions.
- Keep the issue type consistent in both for and against questions, even if the text has multiple issues.
- It's OK to ignore some issues if the text refers to multiple issues.
- Include the general theme of the issue in the questions.

## Question Structure:

- Start with "Explain why..."
- Reference the specific position or viewpoint
- Ask about reasoning, values, or priorities behind that position
- Avoid yes/no formulations

# Output Format:
Respond with a JSON object containing exactly these fields:

{
  "specific_divergent_issue": "[One line description of the specific chosen issue]",
  "for_description": "[Brief description of the first viewpoint/position]",
  "for_question": "Explain why [specific question for holders of the first viewpoint/position]",
  "against_description": "[Brief description of the opposing viewpoint/position]", 
  "against_question": "Explain why [specific question for holders of opposing viewpoint/position]"
}

# Example:

## Input
Title: Heavy horse (General)

Section: Controversy and social acceptance

Text: Raising horses for meat is controversial, with many people opposed to hippophagy, and to the very idea of breeding horses for this purpose. According to Bernadette Lizet, in France, the heavy horse breeders present at the Salon International de l'Agriculture have taken to hiding their motivations from Parisian visitors, citing "passion", without ever mentioning the competition criteria for heavy breeds, or fattening workshops, or "finishing-meat in Italy". City dwellers are indeed shocked by the existence of such breeding, which they readily associate with "barbarism". Jean-Pierre Digard cites the example of a "breeder of splendid heavy horses paralyzed by the fear of having to explain that he was raising horses for slaughter", during an interview on a presentation podium at the 2008 Salon International de l'Agriculture. In the Spanish Basque Country, the marketing of locally-bred foal meat is based on an elaborate commercial strategy. The language is modernized to lessen the emotional impact created by the idea of consuming horse, speaking instead of "foal meat" (Carne de potro), the emotional impact of the word "foal" not being deemed as strong as that of the word "horse". Other controversies concern the abuse that some breeders inflict on heavy foals, and the awarding of breeding premiums to animals in poor health (obese, even lame) to the detriment of working horses, particularly in the Breton and Comtoise breeds.

Issue: This text discusses a **divergent issue**: the ethical and social acceptability of raising horses for meat (hippophagy). The text clearly outlines differing viewpoints – those who oppose it based on ethical concerns and those who engage in the practice, attempting to mitigate negative perceptions. The breeders' attempts to conceal their motivations and the use of euphemistic language demonstrate a clear conflict in values and social acceptance.

## Response
{
  "specific_divergent_issue": "The acceptance of the practice of eating horse meat.",
  "against_description": "Those who raise horses for meat, particularly heavy horse breeders, participate in the practice as a means of livelihood and potentially due to competition criteria within the industry. They attempt to navigate social perceptions by downplaying the association with 'horse' and focusing on terms like 'foal meat'.",
  "against_question": "Explain why, despite the shock of some people like Parisian city dwellers, you believe raising horses for meat is a legitimate and ethical practice and can be justified as a cultural practice, and what steps you take to ensure the welfare of the animals throughout their lives.",
  "for_description": "Those who oppose raising horses for meat believe it is ethically wrong, associating it with 'barbarism' and expressing emotional distress at the idea of consuming horse meat, particularly foals. They are concerned about animal welfare and the potential for abuse.",
  "for_question": "Explain why you believe people are right to be shocked by the practice of hippophagy and that horses should be treated differently from other livestock animals commonly used for food, and what specific aspects of raising horses for meat you find most concerning."
}
"""

In [58]:
def format_section(title, section, text, reasoning):
    return f"Title: {title}\n\nSection: {section}\n\nText: {text}\n\nIssue:\n\n{reasoning}"

In [59]:
def extract_questions(response):
    try:
        data = json.loads(response)
        for_question = data['for_question']
        against_question = data['against_question']
    except (ValueError, KeyError) :
        for_question = ""
        against_question = ""

    return data, for_question, against_question

In [71]:
OUTPUT_JSON_SCHEMA = {
  "type": "object",
  "properties": {
    "specific_divergent_issue": {
        "type": "string"
    },
    "for_description": {
      "type": "string"
    },
    "for_question": {
      "type": "string"
    },
    "against_description": {
      "type": "string"
    },
    "against_question": {
      "type": "string"
    }
  },
  "required": [
    "specific_divergent_issue",
    "for_description",
    "for_question",
    "against_description",
    "against_question"
  ],
  "additionalProperties": False
}

In [72]:
def generate_questions(titles, sections, texts, reasonings):
    messages = [[{"role": "system", "content": system_prompt},
                {"role": "user", "content": format_section(title, section, text, reasoning['divergent_issue_rationale'])}]
                for title, section, text, reasoning in zip(titles, sections, texts, reasonings)]
    
    outputs = llm.chat(messages, SamplingParams(temperature=0.6, top_p=0.9, max_tokens=512, guided_decoding=GuidedDecodingParams(json=OUTPUT_JSON_SCHEMA)))
    responses = [output.outputs[0].text.strip() for output in outputs]
    datas, for_questions, against_questions =  zip(*[extract_questions(response) for response in responses])
    
    return {"questions_json": list(datas), "for_question": list(for_questions), "against_question": list(against_questions)}

In [ ]:
total_count = ds.num_rows
num_steps = 25
step_size = math.ceil(total_count / num_steps)

for step in range(num_steps):
    print(f"Running step: {step}")

    start_i = step * step_size
    end_i = (step+1) * step_size
    

    cls_ds = ds.select(range(start_i, min(end_i, ds.num_rows))).map(generate_questions, batched=True, batch_size=step_size, input_columns=['article_title', 'section_title', 'section_text', 'classification_json'])
    cls_ds.save_to_disk(f"questions-test-{step}.hf")

Running step: 0


Map:   0%|          | 0/7023 [00:00<?, ? examples/s]

Adding requests:   0%|          | 0/7023 [00:00<?, ?it/s]

Processed prompts:   0% 0/7023 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [2]:
import glob

files = glob.glob("questions-*")

In [3]:
files

['questions-0.hf',
 'questions-1.hf',
 'questions-2.hf',
 'questions-3.hf',
 'questions-4.hf',
 'questions-5.hf',
 'questions-6.hf',
 'questions-7.hf',
 'questions-8.hf',
 'questions-9.hf',
 'questions-10.hf',
 'questions-11.hf',
 'questions-12.hf',
 'questions-13.hf',
 'questions-14.hf',
 'questions-15.hf',
 'questions-16.hf',
 'questions-17.hf',
 'questions-18.hf',
 'questions-19.hf',
 'questions-20.hf',
 'questions-21.hf',
 'questions-22.hf',
 'questions-23.hf',
 'questions-24.hf']

In [4]:
processed_ds = []
for f in files:
    processed_ds.append(load_from_disk(f))

In [5]:
combined_ds = concatenate_datasets(processed_ds)
combined_ds.num_rows

175575

In [8]:
combined_ds[1]

{'article_title': 'Heavy horse (General)',
 'url': 'https://en.wikipedia.org/wiki/Heavy_horse_(General)',
 'section_title': 'Controversy and social acceptance',
 'section_text': 'Raising horses for meat is controversial, with many people opposed to hippophagy, and to the very idea of breeding horses for this purpose. According to Bernadette Lizet, in France, the heavy horse breeders present at the Salon International de l\'Agriculture have taken to hiding their motivations from Parisian visitors, citing "passion", without ever mentioning the competition criteria for heavy breeds, or fattening workshops, or "finishing-meat in Italy". City dwellers are indeed shocked by the existence of such breeding, which they readily associate with "barbarism". Jean-Pierre Digard cites the example of a "breeder of splendid heavy horses paralyzed by the fear of having to explain that he was raising horses for slaughter", during an interview on a presentation podium at the 2008 Salon International de l\

In [7]:
print(combined_ds[0]['section_text'])

One of the most notable incidents that occurred regarding the magazine was an unauthorized adaptation of Harlan Ellison 's short story, "A Boy and His Dog", which has been rumored as one of the major factors in the bankruptcy of Warren Publishing. As discussed in the book The Warren Companion, editor Bill Dubay approached writers Gerry Boudreau and Jim Stenstrum about adapting science fiction stories for the magazine. Boudreau asked permission to adapt Ellison's story, and Dubay approved this, without first asking Ellison. When Ellison refused to grant permission, Dubay had artist Alex Niño draw the story anyway, then provided the art to Stenstrum to use as the basis for a new story. The story was published in issue #4, under the title "Mondo Megillah". Despite Stenstrum's reworking of the script, the basic story was still obvious plagiarism and Ellison filed a lawsuit, which he eventually won. Advertised as an adult fantasy magazine, 1984 contained very mature subject matter by the st

In [ ]:
#combined_ds.push_to_hub('amang1802/wikipedia_controversial_abductive_alignment_prompts')